# Practical Exercise 4.04: Inmunization

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Initial data

required_payment = 5000000  # Future value in year 3
target_duration = 3

# Bond durations

duration_A = 1.97
duration_B = 3.80
duration_C = 4.63

# Current bond prices

price_A = 1000
price_B = 1000
price_C = 1000

# Bond Yields (discount rates)

rate_A = 0.03
rate_B = 0.035
rate_C = 0.04

# Define the equations to be solved

def duration_matching_minimize(weights):
    w_A, w_B, w_C = weights


    # The duration of the portfolio should be 3 years.
    duration_constraint = (w_A * duration_A + w_B * duration_B + w_C * duration_C - 3)**2

    # Weights must add up to 100%.

    weights_sum_constraint = (w_A + w_B + w_C - 1)**2
    return duration_constraint + weights_sum_constraint

# Restrictions to ensure that weights are >= 0

constraints = (
    {'type': 'ineq', 'fun': lambda x: x[0]},  # w_A >= 0
    {'type': 'ineq', 'fun': lambda x: x[1]},  # w_B >= 0
    {'type': 'ineq', 'fun': lambda x: x[2]}   # w_C >= 0
)

# Initial assumptions for bond weights A, B and C

initial_weights = [0.33, 0.33, 0.34]

# Solve the equations to find the correct bond weights.

result = minimize(duration_matching_minimize, initial_weights, constraints=constraints)
weights_solution = result.x

# Bond weights

w_A, w_B, w_C = weights_solution


# Calculate the IRR of the portfolio as the weighted average of the bond IRRs.

portfolio_IRR = w_A * rate_A + w_B * rate_B + w_C * rate_C

# Calculate the present value of the liability using the IRR of the portfolio.

def present_value(rate, future_value, years):
    return future_value / (1 + rate)**years

present_value_liability = present_value(portfolio_IRR, required_payment, 3)

# Calculate the amount to invest in each bond

amount_invested_A = w_A * present_value_liability
amount_invested_B = w_B * present_value_liability
amount_invested_C = w_C * present_value_liability

# Calculate the number of units of each bond

no_securities_A = amount_invested_A / price_A
no_securities_B = amount_invested_B / price_B
no_securities_C = amount_invested_C / price_C

# Create the results DataFrame

results = pd.DataFrame({
    'Bond': ['A', 'B', 'C'],
    'Weight': [w_A, w_B, w_C],
    'Amount Invested': [amount_invested_A, amount_invested_B, amount_invested_C],
    'Current Price': [price_A, price_B, price_C],
    'Number of Securities': [round(no_securities_A,0), round(no_securities_B,0), round(no_securities_C,0)],
    'Bond Yield(%)': [rate_A*100, rate_B*100, rate_C*100],
    'Duration': [duration_A, duration_B, duration_C]
})

# Round IRR Bond to four decimal places

results['Bond Yield(%)'] = results['Bond Yield(%)'].apply(lambda x: '{:.2f}'.format(x))

# Add additional portfolio details

portfolio_IRR_rounded = '{:.2f}'.format(portfolio_IRR*100)
present_value_liability_rounded = round(present_value_liability, 2)

# Create an additional row for portfolio details

portfolio_details = pd.DataFrame({
    'Bond': ['Inmunized Portfolio'],
    'Weight': [1.0],
    'Amount Invested': [present_value_liability_rounded],
    'Current Price': ['-'],
    'Number of Securities': ['-'],
    'Bond Yield(%)': [portfolio_IRR_rounded],
    'Duration': [target_duration]
})

# Concatenate the results table with the portfolio details row

final_results= pd.concat([results, portfolio_details], ignore_index=True)

# Display results without scientific notation and with thousands separator

pd.set_option('display.float_format', lambda x: '{:,.2f}'.format(x))
final_results
